In [ ]:
import sklearn
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs


In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [ ]:
jan = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-01-01_2020-01-31.tsv", sep='\t', header=0)

#feb = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByCountry_2020-02-01_2020-02-29.tsv", sep='\t', header=0)

mar = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-03-01_2020-03-31.tsv", sep='\t', header=0)

apr = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-04-01_2020-04-30.tsv", sep='\t', header=0)

may = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-05-01_2020-05-31.tsv", sep='\t', header=0)

jun = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-06-01_2020-06-30.tsv", sep='\t', header=0)

#jul =pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-07-01_2020-07-31.tsv", sep='\t', header=0)

#aug = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-08-01_2020-08-31.tsv", sep='\t', header=0)

#sep = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-09-01_2020-09-30.tsv", sep='\t', header=0)

#oct = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-10-01_2020-10-31.tsv", sep='\t', header=0)

#nov = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-11-01_2020-11-30.tsv", sep='\t', header=0)

#dec = pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2020/QueriesByState_2020-12-01_2020-12-31.tsv", sep='\t', header=0)

jan21= pd.read_csv("https://raw.githubusercontent.com/microsoft/BingCoronavirusQuerySet/master/data/2021/QueriesByState_2021-01-01_2021-01-31.tsv", sep='\t', header=0)


In [ ]:
df = pd.concat([jan, mar, apr, may, jun], ignore_index=True)





In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].str.replace(r"\b\d+(?:\.\d+)?\s+", " ") # remove numbers from strings
    df[text_field] = df[text_field].str.replace(r"[^a-zA-Z]", " ")# Search for all non-letters,  Replace all non-letters with spaces
    return df



In [ ]:
questions = standardize_text(df, "Query")

questions.to_csv("clean_data.csv")
questions.head()

,Date,Query,IsImplicitIntent,State,Country,PopularityScore
0,2020-01-01,coronavirus,False,Michigan,United States,1
1,2020-01-01,coronavirus,False,Pennsylvania,United States,1
2,2020-01-01,p masks,True,Australian Capital Territory,Australia,1
3,2020-01-01,ausw rtiges amt,True,North Rhine-Westphalia,Germany,1
4,2020-01-01,p masks,True,New South Wales,Australia,1


In [ ]:
clean_questions1= pd.read_csv("clean_data.csv")
clean_questions1.tail()



,Unnamed: 0,Date,Query,IsImplicitIntent,State,Country,PopularityScore
2949623,2949623,2020-06-30,nzta,True,Waikato,New Zealand,1
2949624,2949624,2020-06-30,virginia coronavirus,False,Virginia,United States,46
2949625,2949625,2020-06-30,de morgen,True,Brussels Region,Belgium,26
2949626,2949626,2020-06-30,kdhe covid,False,Kansas,United States,2
2949627,2949627,2020-06-30,nationwide building society,True,Lanarkshire,United Kingdom,13


In [ ]:
clean_questions = clean_questions1.loc[clean_questions1.Query.str.contains("how", na=False)]

In [ ]:
corpus = clean_questions.loc[(clean_questions.Country== "United States") & (clean_questions.Country== "United Kingdom")]

In [ ]:
corpus['Query']=corpus['Query'].apply(str)

In [ ]:
clean_questions['PopularityScore'].mean(axis = 0, skipna = True)

nan

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
unique_words = set(all_words)
print(len(unique_words))

9


In [ ]:
vocab_length = 50

In [ ]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[48, 4], [29], [22], [3], [20], [25], [17]]


In [ ]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [ ]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[48  4  0]
 [29  0  0]
 [22  0  0]
 [ 3  0  0]
 [20  0  0]
 [25  0  0]
 [17  0  0]]


In [ ]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 20)             1000      
_________________________________________________________________
flatten (Flatten)            (None, 60)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 1,061
Trainable params: 1,061
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 61        
Total params: 1,061
Trainable params: 1,061
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(padded_sentences, labels, epochs=100, verbose=1) #labels not yet defined

ValueError: ignored